In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
data_train = pd.read_csv('D://kaggle_tweet//train_data.csv', sep="*", engine="python",encoding = 'utf-8')
data_test = pd.read_csv('D://kaggle_tweet//test_data.csv', sep="*", engine="python",encoding = 'utf-8')
label_test = pd.read_csv('D://kaggle_tweet//test_labels.csv', sep=",", engine="python",encoding = 'utf-8')

In [3]:
data_train.head()

,Unnamed: 0,id,keyword,location,text,target
0,0,1,NaN,NaN,deeds reason earthquake may allah forgive,1
1,1,4,NaN,NaN,forest fire near ronge sask canada,1
2,2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,4,7,NaN,NaN,sent photo ruby alaska smoke wildfires pours s...,1


In [4]:
data_test.head()

,Unnamed: 0,id,keyword,location,text
0,0,0,NaN,NaN,happened terrible car crash
1,1,2,NaN,NaN,heard earthquake different cities stay safe ev...
2,2,3,NaN,NaN,forest fire spot pond geese fleeing across str...
3,3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,4,11,NaN,NaN,typhoon soudelor killsin china taiwan


In [5]:
label_test.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [6]:
data_test_completed = pd.merge(data_test,label_test, on='id')

In [7]:
data_test_completed.head()

,Unnamed: 0,id,keyword,location,text,target
0,0,0,NaN,NaN,happened terrible car crash,1
1,1,2,NaN,NaN,heard earthquake different cities stay safe ev...,0
2,2,3,NaN,NaN,forest fire spot pond geese fleeing across str...,1
3,3,9,NaN,NaN,apocalypse lighting spokane wildfires,1
4,4,11,NaN,NaN,typhoon soudelor killsin china taiwan,1


In [8]:
data_test_completed = data_test_completed.drop(data_test_completed.columns[0], axis=1)
data_train = data_train.drop(data_train.columns[0], axis=1)

In [9]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive,1
1,4,NaN,NaN,forest fire near ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,7,NaN,NaN,sent photo ruby alaska smoke wildfires pours s...,1


In [10]:
df = pd.concat([data_train,data_test_completed])

In [11]:
df.shape

(10876, 5)

In [12]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive,1
1,4,NaN,NaN,forest fire near ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,7,NaN,NaN,sent photo ruby alaska smoke wildfires pours s...,1


In [13]:
def convert(lst):  #splits setence to list of words
    return (lst[0].split()) 

In [14]:
L = ["reste ici et ne bouge pas "]
convert(L)

['reste', 'ici', 'et', 'ne', 'bouge', 'pas']

In [15]:
def count_word(df):#create a dictionnary keys : all the words in the text column in the df /values number of iterations
    dic_words = {}
    for i in range(df.shape[0]):
        sentence = convert([str(df.iloc[i]['text'])])
        for word in sentence:
            if word in dic_words.keys():
                dic_words[word] = dic_words[word] + 1
            else:
                dic_words[word] = 1
                
    return dic_words
    
def join_keys(dic,dic_total):
    for key in dic_total.keys():
        if key not in dic.keys():
            dic[key] = 0
    return dic    

In [16]:
df_0 = df[df["target"] == 0]
dic_0 = count_word(df_0)

df_1 = df[df["target"] == 1]
dic_1 = count_word(df_1)

dic_total = count_word(df)


In [17]:
dic_0 = join_keys(dic_0,dic_total)
dic_1 = join_keys(dic_1,dic_total)

In [18]:
dic_total.keys() == dic_1.keys()

True

In [19]:
dic_scores = {}
for key in dic_total.keys():
    dic_scores[key] = (dic_1[key]/dic_total[key]) - (dic_0[key]/dic_total[key])
    

In [20]:
def text_score(df,dic_scores):#add a column of the text score
    dic_res = {}
    dic_res["Score_Sentence"] = []
    for i in range(df.shape[0]):
        res = 0
        sentence = convert([str(df.iloc[i]['text'])])
        for word in sentence:
            res = res + dic_scores[word]
        dic_res["Score_Sentence"].append(res)
                
    df_score = pd.DataFrame(dic_res)
    
    #df_res = pd.concat([df["id"],df_score],axis=1)
    return df.join(df_score)

In [21]:
df_res_train = text_score(data_train,dic_scores)
df_res_test = text_score(data_test_completed,dic_scores)
df_res = text_score(df,dic_scores)

In [22]:
df_res_1 = df_res[df_res["target"] == 1]
df_res_0 = df_res[df_res["target"] == 0]
print("resultats target 0", )
print("moyenne",df_res_0["Score_Sentence"].mean())
print("ecart type",df_res_0["Score_Sentence"].std())
print("         ")
print("         ")
print("         ")
print("resultats target 1", )
print("moyenne",df_res_1["Score_Sentence"].mean())
print("ecart type",df_res_1["Score_Sentence"].std())

resultats target 0
moyenne -3.585804651032129
ecart type 3.3281611732749927
         
         
         
resultats target 1
moyenne 1.4340627957423064
ecart type 3.1999704709623145


In [26]:
df_res_train.to_csv('D://kaggle_tweet//data_train_scores_added.csv',sep = ',' , index = None, header=True)
df_res_test.to_csv('D://kaggle_tweet//data_test_scores_added.csv',sep = ',' , index = None, header=True)